In [1]:
from datasets import load_dataset
from haystack.document_stores import InMemoryDocumentStore
from haystack.nodes import BM25Retriever, PromptNode, PromptTemplate, AnswerParser
from haystack.pipelines import Pipeline

In [ ]:
dataset = load_dataset("bilgeyucel/seven-wonders", split="train")
document_store = InMemoryDocumentStore(use_bm25=True)
document_store.write_documents(dataset)
retriever = BM25Retriever(document_store=document_store)

In [3]:
rag_prompt = PromptTemplate(
    prompt="""Synthesize a comprehensive answer from the following text for the given question.
                             Provide a clear and concise response that summarizes the key points and information presented in the text.
                             Your answer should be in your own words and be no longer than 50 words.
                             \n\n Related text: {join(documents)} \n\n Question: {query} \n\n Answer:""",
    output_parser=AnswerParser(),
)
prompt_node = PromptNode(model_name_or_path="google/flan-t5-large", default_prompt_template=rag_prompt, use_gpu=True)

In [4]:
pipe = Pipeline()
pipe.add_node(component=retriever, name="retriever", inputs=["Query"])
pipe.add_node(component=prompt_node, name="prompt_node", inputs=["retriever"])

In [5]:
output = pipe.run(query="What is the Great Pyramid of Giza?")
print(output["answers"][0].answer)
output = pipe.run(query="Where are the hanging gardens?")
print(output["answers"][0].answer)

Token indices sequence length is longer than the specified maximum sequence length for this model (2916 > 512). Running this sequence through the model will result in indexing errors
The prompt has been truncated from 2916 tokens to 412 tokens so that the prompt length and answer length (100 tokens) fit within the max token limit (512 tokens). Shorten the prompt to prevent it from being cut off
The prompt has been truncated from 2980 tokens to 412 tokens so that the prompt length and answer length (100 tokens) fit within the max token limit (512 tokens). Shorten the prompt to prevent it from being cut off


The Great Pyramid of Giza is the oldest of the Seven Wonders of the Ancient World.
The Hanging Gardens are the only one of the Seven Wonders for which the location has not been definitively established.
